In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import yfinance as yf
from tradingo.api import Tradingo
from tradingo import backtest
from tradingo import analytics
from tradingo import signals
from tradingo.portfolio import position_from_trades
from tradingo.sampling import get_ig_service
from tradingo.plotting import plot_multi
from tradingo import signals
import pandas_market_calendars as pmc
import yfinance as yf
from datetime import time
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

# pd.options.plotting.backend = 'plotly'

pd.options.plotting.backend = 'matplotlib'
# t = Tradingo(name="etft", uri="s3://s3.us-east-1.amazonaws.com:tradingo-store?aws_auth=true&path_prefix=prod", provider="ig-trading", universe='igtrading')
t = Tradingo(name="etft", uri="lmdb:///home/rory/dev/airflow/test/arctic.db", provider="ig-trading", universe='im-assets')

In [ ]:
vix = yf.download("^VIX", start="2017-01-01", end="2024-10-18")

In [ ]:
vixsignal = (vix.ewm(12).mean().Close - vix.ewm(60).mean().Close)#.where(vix.abs() < 10, 10 * np.sign(vix))
vixsignal = vixsignal.where(vixsignal < 10, 10 * np.sign(vixsignal)).where(vixsignal.abs() > 2, 0).shift().tz_localize('utc').reindex(bt_all.index, method='nearest').ffill()

In [ ]:
%pdb on
import matplotlib.pyplot as plt

all_signals = signals.intraday_momentum(
    universe='im-assets',
    provider='ig-trading',
    dry_run=True,
    start_date=pd.Timestamp("2017-01-01 00:00:00+00:00"),
    end_date=pd.Timestamp("2024-10-15 15:00:00+00:00"),
    threshold=1,
    cap=2,
    close_offset_periods=0,
    arctic=t,
    monotonic = True,
).fillna(0.0)

all_signals.index.name = "DateTime"
signal = all_signals["signals/intraday_momentum"]

# avg_spread = (0.5 - (t.prices.ask.close() - t.prices.bid.close()).reindex(all_signals.index).resample(pd.offsets.BDay(1)).median().replace(0.0, np.nan).ffill().rolling(6).mean())

position = ((
    1
    * vixsignal
    # * np.sign(avg_spread.shift()).reindex(all_signals.index, method='nearest').ffill().transpose()
    * -1
    # * vix_indicator().reindex(close.index, method='ffill')
    * signal.ffill().fillna(0.0).transpose()
    ).transpose()
    # .reindex_like(t.portfolio.intraday.rounded.position(), method='ffill')
    .fillna(0.0).round()
)

position = position * pd.Series({"CC.D.NG.UMP.IP": 500, "IX.D.SPTRD.IFS.IP": 1})


bt_all = backtest.backtest(
    # bid=bid_close.dropna(),
    bid_close='prices/bid.close',
    ask_close='prices/ask.close',
    stop_limit=None,
    stop_loss=None,
    portfolio=position,
    # ask=ask_close.dropna(),
    # portfolio=position.round(),
    # portfolio=t.portfolio.intraday.rounded.position(),
    dividends=None,
    universe='im-assets',
    provider='ig-trading',
    name='intraday',
    stage='rounded.position',
    dry_run=True,
    arctic=t,
    start_date=pd.Timestamp("2020-01-01 00:00:00+00:00"),
    end_date=pd.Timestamp("2024-10-16 16:15:00+00:00")
)

bt = bt_all['backtest/portfolio']

(
    bt['total_pnl']
    .resample(pd.offsets.BDay(1))
    .last()
    .plot()
    # .show()
)

data = pd.concat(
    (
        t.prices.mid.close(),
        bt,
        bt_all.droplevel(1, axis=1),
        position.squeeze(),
        all_signals.droplevel(1, axis=1),
    ),
    axis=1
)


# plot_multi(
#     data.loc[slice("2017-12-02. 14:00:00", "2017-12-04")].reset_index(),
#     x='DateTime',
#     y=[
#         'position',
#         # 'total_pnl',
#         # 'signals/intraday_momentum',
#         # 'mid_close',
#         'unrealised_pnl',
#         'signals/intraday_momentum.z_score',
#     ],
#     figsize=(16,7),
#     # vline=0.01,
# )

# # data['net_exposure'].plot()

# plt.axhline(1.0, c='black', linestyle='--')
# plt.axhline(0.0, c='black', linestyle='--')
# plt.axhline(2.0, c='black', linestyle='--')
# plt.axvline(pd.Timestamp("2017-12-04 21:00:00", tz="America/New_York"))
# data.loc[slice("2017-12-02. 14:00:00", "2017-12-04")].reset_index()

bt_all_ = bt_all.resample(pd.offsets.BDay(1)).last()

t.prices.mid.close().loc[date][["CC.D.NG.UMP.IP"]].plot()
bt_all_["backtest/instrument.total_pnl"].plot()
bt_all_["backtest/portfolio"]['total_pnl'].plot()

bt_all_["backtest/instrument.net_position"].rolling(25).max().plot()

In [ ]:
threshold = 1
cap = 2
# z_score = t.signals.intraday_momentum.z_score()
                                                                                        
signal = all_signals['signals/intraday_momentum']                                                       
                                                                                    
# apply caps and threshold                                                          
#                                                                                   
                                                                     

signal.loc["2024-10-14"].plot()#.show()
signal.loc["2024-09-06"].plot()#.show()
signal.loc["2023-11-14"].plot()#.show()

In [ ]:
signal = signal.where(signal.index.to_series().dt.date.eq(signal.index.to_series().dt.date.shift()), 0).where(np.sign(signal.shift()).eq(np.sign(signal)) , 0)
signal.loc["2024-09-06"].plot()
signal.loc["2023-11-14"].plot()

In [ ]:
date = slice("2021-12-15", "2021-12-20")
z_score = all_signals["signals/intraday_momentum.z_score"]
signal.loc[date].plot()
z_score.loc[date].plot()


In [ ]:
%pdb off
t.prices.mid.close()
signal
t.prices.mid.close().loc["2017-01-08 14"]
signal.loc["2018-02-14"]
# t.prices.mid.close().loc["2017-02-14"]
all_signals["signals/intraday_momentum.z_score"]

In [ ]:

signal = signal.where(
   (
      np.sign(signal.shift()).eq(np.sign(signal))
      & signal.index.to_series().dt.date.eq(
         signal.index.to_series().dt.date.shift()
      )
  ),
   0,
)

signal.loc["2024-10-14"].plot()


In [ ]:
def dynamic_floor(series, shift=0):                                                 
    return series.groupby(series.index.date).transform(                             
        lambda i: i.where(                                                          
          ~functools.reduce(                                                      
              operator.and_,                                                      
             (                                                                   
                (np.sign(i.shift(s)).ne(0) & np.sign(i.shift(s)).ne(np.sign(i)))
                for s in range(1, shift)                                        
             ),                                                                  
           ),                                                                      
          0,                                                                      
        )                                                                              
    )                                                                               
                                                                                    
signal = dynamic_floor(signal, 5)                                                   
signal.loc["2024-10-14"].plot()

In [ ]:

(bt_all["backtest/portfolio"]["gross_exposure"]).resample(pd.offsets.BDay(1)).last().dropna().plot()


In [ ]:
t.prices.mid_close()

In [ ]:
avg_spread = (0.5 - (t.prices.ask.close() - t.prices.bid.close()).resample(pd.offsets.BDay(1)).median().replace(0.0, np.nan).ffill().rolling(30).mean())
np.sign(avg_spread.shift())

In [ ]:
df = data.reindex(all_signals.index).resample(pd.offsets.BDay(1)).last().dropna(subset=["mid_close", "total_pnl"])
np.corrcoef(df[df['total_pnl']!=0]['total_pnl'].pct_change().dropna(),
    df[df['total_pnl']!=0]['mid_close'].pct_change().dropna())

In [ ]:
data[(data.gross_exposure!=0) & (data.gross_exposure.notna())]

In [ ]:
date = "2017-04-05"
data[["mid_close",
"unrealised_pnl",
"position",
"signals/intraday_momentum.z_score",
"signals/intraday_momentum"]].query("position!=0").loc[date]

In [ ]:
all_signals['signals/intraday_momentum'].resample(pd.offsets.BDay(1)).min().reset_index().plot.scatter(x="DateTime", y="IX.D.SPTRD.IFS.IP")

In [ ]:
all_signals["signals/intraday_momentum"].loc[date]

In [ ]:
import functools


def dynamic_floor(z_score, shift=0):
    return (
        z_score
        .groupby(z_score.index.date).transform(lambda i: i.where(~((i.notna() & np.sign(i.shift(shift))!=0)  & (np.sign(i.shift(shift)) != np.sign(i))), 0))
    )

z_score = all_signals["signals/intraday_momentum"]


for i in range(0, 1):
    z_score = dynamic_floor(z_score, i)
    z_score.loc[date].plot().show()



In [ ]:
print((bt.total_pnl.resample(pd.offsets.BDay(1)).last().diff()/50_000).std() * np.sqrt(252))
(bt.total_pnl.resample(pd.offsets.BDay(1)).last().diff()/50_000).mean() * 252

In [ ]:
z_score = all_signals["signals/intraday_momentum.z_score"]
z_score = z_score[z_score.squeeze()>0]
signal = all_signals["signals/intraday_momentum"]

z_score.abs().groupby(z_score.tz_convert("America/New_York").index.time).mean().plot()

In [ ]:
(0.05 * bt.query('net_exposure != 0').abs().net_exposure).plot.hist(bins=20)

In [ ]:
def mean_reversion():
    long_mean = close[close.index.time == time(20)].ewm(3).mean().squeeze().rename('long')
    im_mean = close.ewm(6).mean().squeeze().rename('short')
    mr = pd.concat((long_mean, im_mean), axis=1).ffill()
    
    signal = (mr.long - mr.short)/mr.short
    signal = signal * 100
    signal[signal.abs() < 1] = 0
    signal[signal.abs() > 2] = 2

    return signal.rename(close.columns[0]).to_frame()

In [ ]:
print(analytics.sharpe_ratio(
    # t.prices.mid.close().resample(pd.offsets.BDay(1)).last().pct_change()
    bt.total_pnl.resample(pd.offsets.BDay(1)).last().diff()/4_000
))
print(analytics.sharpe_ratio(
    t.prices.mid.close().squeeze().resample(pd.offsets.BDay(1)).last().pct_change()
    # bt.total_pnl.resample(pd.offsets.BDay(1)).last().diff()/4_000
))

In [ ]:
print(analytics.sharpe_ratio(bt.total_pnl.resample(pd.offsets.BDay(1)).last().diff()/4_000))
print(analytics.omega_ratio(bt.total_pnl.resample(pd.offsets.BDay(1)).last().diff()/4_000))

In [ ]:
date = "2024-08-05"
bt_all["backtest/instrument.total_pnl"].loc[date].diff().cumsum().plot()
bt_all["backtest/instrument.net_position"].loc[date].plot()
all_signals["signals/intraday_momentum.z_score"].loc[date].plot()
all_signals["signals/intraday_momentum"].loc[date].plot()
# data["mid_close"].loc[date].plot()

In [ ]:
bt_all["backtest/instrument.total_pnl"].squeeze().resample(pd.offsets.BDay(1)).last().diff().idxmin()

In [ ]:
date = "2018-10-14"
bt.total_pnl.resample(pd.offsets.BDay(1)).last() #.diff()
bt.total_pnl.diff().loc[date].cumsum().plot().show()
t.prices.mid.close().loc[date].plot().show()
# bt.net_exposure.diff().loc["2024-09-04"].cumsum().plot()
bt_all["backtest/instrument.net_position"].diff().loc[date].cumsum().plot()
# bt.total_pnl.loc["2024-10-15"].plot()




In [ ]:
bt_all['backtest/portfolio']

In [ ]:
bt.query('net_exposure.groupby(index.dt.date).cummax()!=0.0').total_pnl.resample(pd.offsets.BDay(1)).last().diff().plot.hist(bins=100)

In [ ]:
bt.total_pnl.resample(pd.offsets.MonthEnd(1)).last().plot()

In [ ]:
t.signals.intraday_momentum().loc["2024-10"].plot()

In [ ]:
bt.total_pnl.resample(pd.offsets.BDay(1)).last().diff().idxmax()

In [ ]:
# (t.backtest.intraday.rounded.position.portfolio().query("net_exposure.shift()!=0.0").loc["2024-10-14"].unrealised_pnl).plot()

bt.total_pnl.loc["2024-04-04"].diff().cumsum().plot()



In [ ]:
pf_tm1 = pf_lib.read("ig-trading.igtrading.intraday.rounded.position", as_of=-2).data
pf_t = pf_lib.read("ig-trading.igtrading.intraday.rounded.position", as_of=-1).data

In [ ]:
(pf_tm1.reindex(pf_t.index, method='ffill') - pf_t).iloc[-1:,:]

In [ ]:
t.trades.intraday.rounded.position()

In [ ]:
t.backtest.intraday.rounded.position.portfolio().total_pnl.resample(pd.offsets.BDay(1)).last().plot()

In [ ]:
signal = t.signals.intraday_momentum()

In [ ]:
t.backtest.intraday.rounded.position.portfolio().total_pnl.loc["2024-10"].plot()

In [ ]:
signal = t.signals.intraday_momentum()
prices = t.prices.mid.close().reindex(signal.index)

previous_close = prices.groupby(prices.index.date).last().shift(1)
previous_close.index = pd.to_datetime(previous_close.index).tz_localize('utc')
previous_close = previous_close.reindex(prices.index, method='ffill')

In [ ]:
close_returns = (prices - previous_close)/previous_close

data = pd.concat((close_returns.squeeze().rename('returns'), signal.squeeze().rename('signal')), axis=1)#
data.plot.scatter(x='returns', y='signal')

In [ ]:
data.groupby(data.index.date).first().plot.scatter(x='returns', y='signal')

In [ ]:
prices.resample(pd.offsets.BDay(1)).last().dropna().plot()

In [ ]:
t.instruments.igtrading().transpose()

In [ ]:
from tradingo.sampling import get_ig_service

In [ ]:
service = get_ig_service()

In [ ]:
t.signals.intraday_momentum.z_score.tail(200).plot()

In [ ]:
bt = t.backtest.intraday.rounded.position.portfolio()

In [ ]:
bt.total_pnl.resample(pd.offsets.BDay(1)).last().plot()

In [ ]:
t.prices.mid.close().resample(pd.offsets.BDay(1)).last().plot()

In [ ]:
bt = t.backtest.intraday.rounded.position.portfolio()

In [ ]:
bt.total_pnl.diff().loc["2024-10-14"].cumsum().plot()

In [ ]:
daily_returns = bt.total_pnl.resample(pd.offsets.BDay(1)).last().diff()

In [ ]:
(daily_returns.resample(pd.offsets.MonthEnd(1)).sum())

In [ ]:
z_score = t.signals.intraday_momentum.z_score()
short_vol = t.signals.intraday_momentum.short_vol()
long_vol = t.signals.intraday_momentum.long_vol()
previous_close_px = t.signals.intraday_momentum.previous_close_px()
mid_close = t.prices.mid.close()

In [ ]:
data.reindex(z_score.index).loc["2017-12-04"]

In [ ]:
cal = pmc.get_calendar("NYSE")
schedule = cal.schedule(start_date=mid_close.index[0], end_date=mid_close.index[-1])
trading_index = pmc.date_range(schedule, frequency="15min").intersection(mid_close.index)
mid_close = mid_close.reindex(trading_index)

In [ ]:
mid_close.groupby(mid_close.index.date).last().loc["2023-09-11"]

In [ ]:
rounded_position = t.portfolio.intraday.rounded.position().resample(pd.offsets.BDay(1)).mean()

In [ ]:
backtest = t.backtest.intraday.rounded.position.portfolio()

In [ ]:
backtest[backtest.net_exposure!=0]